Imports:

In [217]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import qiskit 
from qiskit_algorithms.optimizers import SPSA
from models import CNN_Simple as CNN_Simple
import models
import torchvision
import torchvision.transforms as transforms

Global Variables:

In [218]:
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False)

max = 10000000

data = training_set
target = training_set

Code:

In [219]:
def loss_f(weights = [], target = torch.tensor([0])):
    model = CNN_Simple()
    
    new_dict = {}
    for name, param in model.named_parameters():
        new_dict[name] = param.data.tolist()
    model.load_state_dict(new_dict)
    
    prediction = model.forward(data)
    print(torch.nn.CrossEntropyLoss(prediction, target))
    return torch.nn.CrossEntropyLoss(prediction, target)

In [220]:
class ShowSteps:
  def __init__(self):
    self.losses = []

  def callback(self, n, weights, loss_f, step_size, accepted):
    self.losses.append(loss_f)

In [231]:
steps = ShowSteps()
model = CNN_Simple()
our_spsa = SPSA(maxiter = max, callback = steps.callback) 
param =[elem.detach().numpy() for elem in model.parameters()]
#print(*model.parameters())
#for i in range(len(param)):
 #   print(len(param[i]))

#need too fix:
result = our_spsa.minimize(model, param)
print("Minimum =", result.x)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.